In [1]:
# [DONE] - Define spix_size based on image resolution
# [DONE] - Optical Flow averagings
# [TODO] - Check with predicted optical flow

import os
import torch
from torch.utils import data
from model.SSN import create_ssn_net, Loss, SSN, crop_like, superpixel_flow
from PIL import Image
from model.util import get_spixel_image
import sys
import numpy as np
from tqdm import tqdm
import cv2
import imageio
import matplotlib.pyplot as plt
# sys.path.append('')
from connectivity import enforce_connectivity

from PIL import Image
sys.path.append('RAFT/core')
import argparse
import os
import time
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


from glob import glob 
from skimage.color import rgb2lab
from skimage.util import img_as_float
from skimage import io
import IO
from dataset import transform_and_get_image,PixelFeature, convert_label, transform_and_get_spixel_init, convert_index, Resize

import datasets as datasets
from raft import RAFT
from utils.utils import InputPadder, forward_interpolate

os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def compute_spixels(root, num_spixel, num_steps, pre_model, out_folder):

    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
        # os.makedirs(out_folder+'png')
        # os.makedirs(out_folder + 'mat')

    dtype = 'test'
    dataloader = data.DataLoader(Dataset_T( root = root, num_spixel=num_spixel),
                        batch_size=1, shuffle=False, num_workers=1)
    
    model = create_ssn_net(num_spixels=num_spixel, num_iter=num_steps, num_spixels_h=10, num_spixels_w=10, dtype=dtype, ssn=0)
    model = torch.nn.DataParallel(model)
    
    if pre_model is not None:
        if torch.cuda.is_available():
            model.load_state_dict(torch.load(pre_model))
        else:
           model.load_state_dict(torch.load(pre_model, map_location='cpu'))
    else:
        raise ('no model')
    criten = Loss()
    device = torch.device('cpu')
    if torch.cuda.is_available():
        model.cuda()
        device = torch.device('cuda')
    outputs = []
    for iter, [inputs, num_h, num_w, init_index, cir_index, p2sp_index_, invisible, file_name] in enumerate(dataloader):
        with torch.no_grad():
            img = inputs['img'].to(device)
            label = inputs['label'].to(device)
            problabel = inputs['problabel'].to(device)
            num_h = num_h.to(device); num_w = num_w.to(device)
            init_index = [x.to(device) for x in init_index]
            cir_index = [x.to(device) for x in cir_index]
            p2sp_index_ = p2sp_index_.to(device)
            invisible = invisible.to(device)

            recon_feat2, recon_label, new_spix_indices = model(img, (p2sp_index_, invisible, init_index, cir_index, problabel, num_h,
                                             num_w, device))
            # loss, loss_1, loss_2 = criten(recon_feat2, img, recon_label, label)
            # print(loss)
            given_img = np.asarray(Image.open(file_name[0]))
            h, w = given_img.shape[0], given_img.shape[1]
            new_spix_indices = new_spix_indices[:, :h, :w].contiguous()
            
            
            spix_index = new_spix_indices.cpu().numpy()[0]
            spix_index = spix_index.astype(int)
            outputs.append(spix_index)

            if enforce_connectivity:
                segment_size = (given_img.shape[0] * given_img.shape[1]) / (int(num_h*num_w) * 1.0)
                min_size = int(0.06 * segment_size)
                max_size = int(3 * segment_size)
                spix_index = enforce_connectivity(spix_index[np.newaxis, :, :], min_size, max_size)[0]
            # given_img_ = np.zeros([spix_index.shape[0], spix_index.shape[1], 3], dtype=np.int)
            # h, w = given_img.shape[0], given_img.shape[1]
            # given_img_[:h, :w] = given_img

            print("output =", spix_index.shape)
            counter_image = np.zeros_like(given_img)
            counter_image = get_spixel_image(counter_image, spix_index)
            spixel_image = get_spixel_image(given_img, spix_index)

            imgname = file_name[0].split('/')[-1][:-4]
            out_img_file = out_folder + imgname + '_bdry_.jpg'
            imageio.imwrite(out_img_file, spixel_image)

            # out_file = out_folder + imgname + '.npy'
            # np.save(out_file, spix_index)

            # validation code only for sp_pix 400
            # out_file_mat = out_folder + 'mat/'+ imgname + '.mat'
            # scio.savemat(out_file_mat, {'segs': spix_index})

            # out_count_file = out_folder + 'png/' + imgname + '.png'
            # imageio.imwrite(out_count_file, counter_image)
            print(iter)
    return outputs



alt cuda corr not found


In [2]:
# outputs = compute_spixels(args.root, args.n_spixels, args.num_steps,args.pre_dir, args.result_dir)

In [3]:
class ARGS:
    def __init__(self):
        self.n_spixels=100
        self.num_steps=10
        self.result_dir='./results/pix/'
        self.pre_dir='./45000_0.527_model.pt'
        self.root = 'data'
args = ARGS()


def imgtensor2np(img):
    return img.permute(1,2,0).detach().cpu().numpy()
def to_device(args, device):
    args_out = []
    for arg in args:
        if isinstance(arg, list):
            arg = [ elem.to(device) for elem in arg ]
        else:
            arg = arg.to(device)
        args_out.append(arg)
    return args_out

def segmentfromLabels(given_img, new_spix_indices, num_h,num_w, enforce_connectivity=False):
    h, w = given_img.shape[0], given_img.shape[1]
    new_spix_indices = new_spix_indices[:, :h, :w].contiguous()
    spix_index = new_spix_indices.cpu().numpy()[0]
    spix_index = spix_index.astype(int)

    if enforce_connectivity:
        segment_size = (given_img.shape[0] * given_img.shape[1]) / (int(num_h*num_w) * 1.0)
        min_size = int(0.06 * segment_size)
        max_size = int(3 * segment_size)
        spix_index = enforce_connectivity(spix_index[np.newaxis, :, :], min_size, max_size)[0]

    return  get_spixel_image(given_img, spix_index)


def ssn_preprocess(im, gtseg, k = 100):
    h, w, _ = im.shape
    # set super pixel stats
    # k =  num_spixel
    k_w = int(np.floor(np.sqrt(k * w / h)))
    k_h = int(np.floor(np.sqrt(k * h / w)))
    spixel_height = h / (1. * k_h)
    spixel_width = w / (1. * k_w)
    out_height = int(np.ceil(spixel_height) * k_h)
    out_width = int(np.ceil(spixel_width) * k_w)

    # process image.
    out_img = transform_and_get_image(im, k, [out_height, out_width])
    pos_scale = 2.5 * max(k_h/out_height, k_w/out_width)
    out_img = PixelFeature(out_img, color_scale=0.26, pos_scale=pos_scale, type='RGB_AND_POSITION') ## add xy information

    # initial superpixels
    out_spixel_init, feat_spixel_init, spixels_h, spixels_w  =  transform_and_get_spixel_init(k, [out_height, out_width]) # changes with k
    init, cir, p2sp_index_, invisible = convert_index(spixels_w, spixels_w * spixels_h, feat_spixel_init)
    invisible = invisible.astype(np.float)

    args = [spixels_h, spixels_w, init, cir, p2sp_index_, invisible]
    if gtseg is None:
        rgbxy = np.transpose(out_img[0], [2, 0, 1]).astype(np.float32)
        return rgbxy, args

    # process labels
    if gtseg is not None:
        inputs = {}
        inputs['input_im'] = np.transpose(out_img[0], [2, 0, 1]).astype(np.float32)
        inputs['spixel_init'] = out_spixel_init[0].astype(np.float32)
        inputs['feat_spixel_init'] = feat_spixel_init[0].astype(np.float32)

        gtseg_ = np.ones_like(out_img[0, :, :, 0]) * 49
        gtseg_[:h, :w] = gtseg
        label_cropped, problabel_cropped = convert_label(gtseg_)
        label_cropped = np.expand_dims(np.expand_dims(label_cropped, axis=0), axis=0)
        inputs['label'] = label_cropped[0]
        inputs['problabel'] = problabel_cropped[0]
        return inputs, args


In [6]:
# DATA LOADER FOR SSN

class DataTemplate(data.Dataset):
    def __init__(self, root = "/home/gokul/courses/CMSC828I/sinteldata/MPI-Sintel-complete", mode = 'train', shape = (128,256)):

        # self.imfiles = sorted(glob(root + f'/images/{mode}/*.jpg')); self.gtfiles = sorted(glob(root + f'/groundTruth/{mode}/*.mat'))
        self.imfiles = sorted(glob(root + f'/{mode}ing/clean/alley_2/*.png' ))
        self.flofiles = sorted(glob(root +f'/{mode}ing/flow/alley_2/*.flo') )
        assert len(self.imfiles)-1 == len(self.flofiles), f' {len(self.imfiles), len(self.flofiles)-1}'
        self.resize = Resize(shape)

    def __getitem__(self, i):
        # load image and GT segment
        imfile1,imfile2, flofile = self.imfiles[i],self.imfiles[i+1], self.flofiles[i]        
        im1,im2, flo = rgb2lab(img_as_float(io.imread(imfile1)) ),rgb2lab(img_as_float(io.imread(imfile2)) ), IO.read(flofile)
        im1,im2, flo = self.resize(im1, im1.shape[:2]),self.resize(im2, im2.shape[:2]), self.resize(flo, flo.shape[:2])
        h,w = im1.shape[:2]

        k = int(0.5 * (h*w)//25 )
        # print(k)
        ssn_inputs, ssn_args = ssn_preprocess(im1, None, k )
        im1 = np.transpose(im1, [2, 0, 1]).astype(np.float32)
        im2 = np.transpose(im2, [2, 0, 1]).astype(np.float32)
        flo = np.transpose(flo, [2, 0, 1]).astype(np.float32)
        return [im1, im2], flo, ssn_inputs, ssn_args

    def __len__(self):
        return len(self.flofiles)

dataset = DataTemplate()
dataloader= data.DataLoader(dataset,batch_size=1, shuffle=False, num_workers=1)


In [8]:
# predict optical flow and segment
class MODELARGS:
    def __init__(self):
        self.model = "./RAFT/raft-kitti.pth"    
        self.small= False; self.mixed_precision = True; 
        self.alternate_corr=False; self.dropout = 0
modelargs = MODELARGS()

# slic layer
SSNLayer = SSN(args.pre_dir, spixel_size=(5,5),dtype = 'layer', device = device)
# flow network
net = torch.nn.DataParallel(RAFT(modelargs))
net.load_state_dict(torch.load(modelargs.model))
net = net.module.to(device)
net.eval()

args.savefolder = 'sintelsegflow'
IO.foldercheck(f'{args.savefolder}/gt')
IO.foldercheck(f'{args.savefolder}/pred')

for idx, data_sample in tqdm(enumerate(dataloader)):
    [im1, im2], flow, ssn_input, ssn_params = data_sample
    im1, im2, flow = im1.to(device), im2.to(device), flow.to(device)


    padder = InputPadder(im1.shape, mode='sintel')
    input1, input2 = padder.pad(im1, im2)

    flow_low, flow_pr = net(input1, input2, iters=24, test_mode=True)

    ssn_input = ssn_input.to(device)  
    ssn_params = to_device(ssn_params, device)
    ssn_params.extend([None])
    _, spix_indices = SSNLayer(ssn_input, ssn_params)    

    spix_indices = crop_like(spix_indices.unsqueeze(1), im1)
    segflow_GT, _ = superpixel_flow( flow.clone(), spix_indices)
    segflow_pred, _ = superpixel_flow( flow_pr.clone(), spix_indices)    

    
    flowrgb_GT = IO.visualize_flow( imgtensor2np(segflow_GT[0]) ) 
    flowrgb_pred = IO.visualize_flow(imgtensor2np(flow_pr[0]))
    cv2.imwrite(f'{args.savefolder}/gt/{str(idx).zfill(2)}.png', flowrgb_GT)
    cv2.imwrite(f'{args.savefolder}/pred/{str(idx).zfill(2)}.png', flowrgb_pred)


49it [00:56,  1.15s/it]
